In [16]:

from minio import Minio
from minio.error import S3Error
import os
from dotenv import load_dotenv

load_dotenv()


minio_endpoint = "localhost:9000"
access_key =os.getenv('MINIO_ACCESS_KEY')
secret_key = os.getenv('MINIO_SECRET')
bucket_name = os.getenv('MINIO_BUCKET_NAME')

In [15]:
# def create_minio_client(endpoint, access_key, secret_key, secure=False):
#     return Minio(endpoint, access_key=access_key, secret_key=secret_key, secure=secure)
def create_minio_client(endpoint, access_key, secret_key, secure=False):
    client = Minio(endpoint, access_key=access_key, secret_key=secret_key, secure=secure)
    print(f"Created Minio client: {client}")
    return client

def ensure_bucket(client, bucket_name):
    if not client.bucket_exists(bucket_name):
        client.make_bucket(bucket_name)
        print(f"Created bucket: {bucket_name}")

In [17]:
# client = Minio('localhost:9000', access_key=access_key, secret_key=secret_key, secure=False)
# client



def create_minio_client():
    client = Minio(minio_endpoint, access_key=access_key, secret_key=secret_key, secure=False)
    return client

In [18]:
conn = create_minio_client()
conn

In [21]:
#MINIO_ENDPOINT = "minio:9000"


#client = create_minio_client(MINIO_ENDPOINT,access_key, secret_key)
bucket_name = "rogerlake"
ensure_bucket(conn, bucket_name)


S3Error: S3 operation failed; code: InvalidAccessKeyId, message: The Access Key Id you provided does not exist in our records., resource: /rogerlake, request_id: 182BD160CE4C2162, host_id: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8, bucket_name: rogerlake